# Convolutional Neural Network

### Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir dataset
!tar -xvzf '/content/drive/MyDrive/college_map_view_dataset.tar.gz' -C '/content/dataset' | wc -l && ls

### Importing the libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

### Verifying that GPU is enabled

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
#tf.__version__
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# GPU vs CPU
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6466868399999157
GPU (s):
0.04834024900003442
GPU speedup over CPU: 75x


## Data Preprocessing

### Checking for corrupt images  

In [9]:
import cv2
import imghdr
import os

for dir_path, dirs, files in os.walk("/content/dataset/college_map_view_dataset/Train"):
  for file_name in files:
    image_path = os.path.join(dir_path, file_name)
    image = cv2.imread(image_path)
    img_type = imghdr.what(image_path)
    if img_type != "png":
      print("removing image from" + image_path + "...")
      os.remove(image_path)
      print("removed image")

for dir_path, dirs, files in os.walk("/content/dataset/college_map_view_dataset/Test"):
  for file_name in files:
    image_path = os.path.join(dir_path, file_name)
    image = cv2.imread(image_path)
    img_type = imghdr.what(image_path)
    if img_type != "png":
      print("removing image from" + image_path + "...")
      os.remove(image_path)
      print("removed image")

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   #shear_range = 0.2,
                                   #zoom_range = 0.2,
                                   #horizontal_flip = True
                                   )
training_set = train_datagen.flow_from_directory('/content/dataset/college_map_view_dataset/Train',
                                                 target_size = (120, 120),
                                                 batch_size = 32,
                                                #  color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 20170 images belonging to 20 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/dataset/college_map_view_dataset/Test',
                                            target_size = (120, 120),
                                            batch_size = 32,
                                            # color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 8630 images belonging to 20 classes.


## Building the CNN

In [5]:
#Initialising the CNN
cnn = tf.keras.models.Sequential()

#Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[120, 120, 3]))

#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Flatenning
cnn.add(tf.keras.layers.Flatten())

#Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Output Layer
cnn.add(tf.keras.layers.Dense(units=20, activation='softmax'))

## Training the CNN

### Compiling the CNN

In [6]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [7]:
cnn.fit(x = training_set,
        validation_data = test_set,
        epochs = 10,
        steps_per_epoch = 20170//32,
        validation_steps = 8631//32) 

Epoch 1/10
630/630 [==============================] - 44s 65ms/step - loss: 1.5509 - accuracy: 0.5399 - val_loss: 0.1944 - val_accuracy: 0.9329
Epoch 2/10
630/630 [==============================] - 40s 63ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.1746 - val_accuracy: 0.9435
Epoch 3/10
630/630 [==============================] - 40s 64ms/step - loss: 8.5127e-04 - accuracy: 1.0000 - val_loss: 0.1728 - val_accuracy: 0.9420
Epoch 4/10
630/630 [==============================] - 40s 64ms/step - loss: 3.2491e-04 - accuracy: 1.0000 - val_loss: 0.1773 - val_accuracy: 0.9377
Epoch 5/10
630/630 [==============================] - 40s 64ms/step - loss: 1.4635e-04 - accuracy: 1.0000 - val_loss: 0.1855 - val_accuracy: 0.9468
Epoch 6/10
630/630 [==============================] - 40s 64ms/step - loss: 2.8863e-05 - accuracy: 1.0000 - val_loss: 0.2798 - val_accuracy: 0.9337
Epoch 7/10
630/630 [==============================] - 40s 63ms/step - loss: 8.4498e-06 - accuracy: 1.0000 - val_loss: 0.

## Predictions

In [13]:
import numpy as np
from keras.preprocessing import image
for dir_path, dirs, files in os.walk("/content/Test"):
  for file_name in files:
    test_image = image.load_img(dir_path + "/" + file_name, target_size = (120, 120))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    # print(training_set.class_indices)
    # print(result[0])
    # res = dict(reversed(list(training_set.class_indices.items())))
    for i, v in enumerate(result[0]):
      if(v == 1.0):
        for key, val in training_set.class_indices.items():
          if(val == i):
            print("\nResult for " + file_name)
            print("Class: " +  key)
            break
        break


Result for Screenshot 2021-03-24 150512.png
Class: 20

Result for Screenshot 2021-03-24 150153.png
Class: 19

Result for Screenshot 2021-03-24 145449.png
Class: 10


In [ ]:
# import numpy as np
# from keras.preprocessing import image
# path = "/content/Screenshot 2021-03-24 132033.png"
# test_image = image.load_img(path,
#                             # color_mode = "grayscale",
#                             target_size = (120, 120))
# test_image = image.img_to_array(test_image)
# test_image = np.expand_dims(test_image, axis = 0)
# result = cnn.predict(test_image)
# print(training_set.class_indices)
# print(result[0])